In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import sparse
from tqdm.notebook import tqdm

%matplotlib inline

In [2]:
def MNAP(size=20):
    '''
    MNAP(size=20)
        Создаёт метрику под <size> сделанных предсказаний.
        
        Parameters
        ----------
        size : int, default=20
            Размер рекомендованной выборки для каждого пользователя
        
        Returns
        -------
        func(pd.DataFrame, pd.DataFrame) -> float
            Функция, вычисляющая MNAP.
        
    '''
    
    assert size >= 1, "Size must be greater than zero!"
    
    def metric(y_true, predictions, size=size):
        '''
        metric(y_true, predictions, size=size)
            Метрика MNAP для двух перемешанных наборов <y_true> и <y_pred>.
            
            Parameters
            ----------
            y_true : pd.DataFrame
                DataFrame с колонками <user_id> и <target>. 
                В <target> содержится список настоящих org_id, посещённых пользователем. 
                
            predictions : pd.DataFrame
                DataFrame с колонками <user_id> и <target>. 
                В <target> содержится список рекомендованных для пользователя org_id.
                
            Returns
            -------
            float 
                Значение метрики.
        '''
        
        y_true = y_true.rename({'target': 'y_true'}, axis='columns')
        predictions = predictions.rename({'target': 'predictions'}, axis='columns')
        
        merged = y_true.merge(predictions, left_on='user_id', right_on='user_id')
    
        def score(x, size=size):
            '''
            Вспомогательная функция.
            '''
            
            
            y_true = x[1][1]
            predictions = x[1][2][:size]
            
            weight = 0
            
            inner_weights = [0]
            for n, item in enumerate(predictions):
                inner_weight = inner_weights[-1] + (1 if item in y_true else 0)
                inner_weights.append(inner_weight)
            
            for n, item in enumerate(predictions):                
                if item in y_true:
                    weight += inner_weights[n + 1] / (n + 1)
                    
            return weight / min(len(y_true), size)
    
        return np.mean([score(row) for row in merged.iterrows()])
    
        
    return metric


def print_score(score):
    print(f"Score: {score*100.0:.2f}")
    
    
N = 20
MNAP_N = MNAP(N)

In [3]:
to_list = lambda rubrics: [int(rubric) for rubric in str(rubrics).split(' ')]
def apply_to_columns(df, columns, func=to_list):
    for column in columns:
        df.loc[~df[column].isnull(), column] = df.loc[~df[column].isnull(), column].apply(func)

In [4]:
test_users = pd.read_csv("Data/df_test_users.csv")

In [12]:
apply_to_columns(test_users, ["target"])

In [14]:
test_users.head()

,user_id,target
0,10003390898943363405,[2254640610021140037]
1,10003459888618360633,[15317743892392503543]
2,10003578788635255367,[9798128123859696444]
3,10006212098254535703,[4480775814859463381]
4,10006413168486885833,[8022186758461891078]


In [20]:
test_prediction = pd.read_csv("Data/df_test_predict.csv")

In [21]:
apply_to_columns(test_prediction, ["target"])

In [22]:
test_prediction.head()

,user_id,target
0,10003390898943363405,"[12046097390037935713, 5710441047385192800, 14..."
1,10003459888618360633,"[9104453017196776235, 13573322486152844808, 57..."
2,10003578788635255367,"[9104453017196776235, 13573322486152844808, 57..."
3,10006212098254535703,"[12046097390037935713, 5710441047385192800, 14..."
4,10006413168486885833,"[12046097390037935713, 5710441047385192800, 14..."


In [23]:
print_score(MNAP_N(test_users, test_prediction))

Score: 4.17
